In [79]:
from CARP_solver import *
from cProfile import run

In [80]:
info = read_file("../datasets/egl-s1-A.dat")
gv.init(info)

In [81]:
def hamming_distance(s1: Solution, s2: Solution):
    def deadhead(solu: Solution):
        deadhead_link = []
        for route in solu.routes:
            for idx in range(len(route.tasks) - 1):
                deadhead_link.append((route[idx].t, route[idx + 1].s))

        return deadhead_link

    m = min(len(s1.routes), len(s2.routes))
    n = gv.num_tasks
    common = len(set(deadhead(s1)).intersection(set(deadhead(s2))))
    return n + m - common


def pop_dist_rank(pop):
    """计算每个solu在pop中的平均距离，倒序排序，越大排名越高"""
    pop_size = len(pop)  # ! 最后要减一
    dist_matrix = np.zeros((pop_size, pop_size), dtype=int)
    for i in range(pop_size):
        for j in range(i + 1, pop_size):
            dist = hamming_distance(pop[i], pop[j])
            dist_matrix[i, j] = dist_matrix[j, i] = dist
    
    # 添加随机值[0,1)进行排序
    return np.argsort(-dist_matrix.sum(axis=0))


def QDNS_update_pop(pop, solu):
    if not solu.feasible():
        return False

    pop_size = len(pop)
    pop.append(solu)

    cost_rank = np.argsort(np.array([solu.cost for solu in pop]) + np.random.rand(pop_size + 1))
    dist_rank = pop_dist_rank(pop)
    alpha = 0.6
    QDF = [alpha * cost_rank[i] + (1 - alpha) * dist_rank[i] for i in range(pop_size + 1)]
    # 将要被抛弃的pop的idx
    worst = np.argmax(QDF)
    # print(list(map(lambda x: x.cost, pop)))
    # print(QDF, worst)
    pop.pop(worst)  # 直接在原来的pop上改
    if worst == pop_size:
        # 弹出了新加的solu
        return False

    return True
    
    

In [82]:
def main(pop_size, timeout):
    pop = init_pop(pop_size)
    best_solu = min(pop, key=lambda x: x.eval())
    best_eval = best_solu.eval()
    tabu_list = []
    max_len_tabu = 30

    start = end = perf_counter()
    while end - start < timeout:
        # crossover
        co = sample(pop, k=2)
        new_solu = Solution.crossover(co[0], co[1])

        # 生成一个随机数，判断应该进行怎样improve
        ty = randint(0, 2)
        if ty == 0:
            pass
        elif ty == 1:
            # single insertion
            new_solu = single_insert(new_solu, best_eval, tabu_list, 10)
            pass

        
        elif ty == 2:
            # merge split
            new_solu = merge_split(new_solu)
        
        if new_solu is not None and new_solu not in tabu_list:
            pop = update_pop(pop, new_solu)
            # new_solu.assert_demand()
        
            tabu_list.append(new_solu)

        bfs = best_feasible_solu(pop)
        eval = bfs.eval()
        if eval < best_eval:
            best_eval = eval
            print(eval)

        # 删除禁忌表中留存时间过长的解
        if len(tabu_list) >= max_len_tabu:
            tabu_list.pop(0)

        end = perf_counter()
    
    return best_feasible_solu(pop)

In [83]:
# run("print(main(20, 5))")
# best = main(20, 100)

In [84]:
best

solution cost:6082, feasible
{'tasks': [(0, 0): 0, (106, 105): 28, (104, 103): 6, (96, 97): 3, (96, 95): 17, (95, 94): 8, (54, 139): 14, (48, 47): 10, (24, 23): 10, (24, 26): 19, (26, 27): 6, (27, 28): 5, (33, 44): 4, (72, 71): 18, (71, 70): 9, (68, 66): 4, (66, 65): 33, (1, 116): 16, (0, 0): 0], 'cost': 1134, 'remain_cap': 0}
{'tasks': [(0, 0): 0, (42, 43): 31, (10, 26): 50, (27, 29): 5, (29, 31): 28, (23, 19): 33, (21, 19): 10, (11, 10): 25, (32, 138): 13, (63, 64): 15, (0, 0): 0], 'cost': 1083, 'remain_cap': 0}
{'tasks': [(0, 0): 0, (0, 115): 18, (113, 112): 12, (112, 111): 4, (111, 109): 13, (109, 110): 7, (109, 106): 8, (113, 117): 58, (123, 125): 11, (125, 129): 35, (118, 116): 24, (116, 115): 19, (0, 0): 0], 'cost': 499, 'remain_cap': 1}
{'tasks': [(0, 0): 0, (67, 66): 9, (68, 70): 6, (32, 10): 24, (8, 7): 37, (5, 4): 26, (5, 7): 10, (7, 10): 48, (45, 42): 5, (76, 77): 15, (83, 84): 10, (0, 0): 0], 'cost': 1001, 'remain_cap': 20}
{'tasks': [(0, 0): 0, (78, 77): 12, (72, 43): 36,

In [85]:
sorted(zip([1, 2, 3], [5, 6, 7]))

[(1, 5), (2, 6), (3, 7)]

In [91]:
import copy
import sys
import time
import random

import numpy as np

MAX = 1e15

exchange_rate = 0.3
swap_rate = 0.3
ran_choice_rate = 0.3


class edge():
    def __init__(self, n1, n2, cost, demand):
        self.n1 = n1
        self.n2 = n2
        self.cost = cost
        self.demand = demand

    def exchange(self):
        tmp = self.n1
        self.n1 = self.n2
        self.n2 = tmp

    def __str__(self):
        return '(' + str(self.n1) + ',' + str(self.n2) + ')'

    def str_r(self):
        return '(' + str(self.n2) + ',' + str(self.n1) + ')'


class result():
    def __init__(self, p, path):
        self.p = p
        self.path = path
        self.cost = p.get_cost(path)

    def breakup(self):
        return [random.sample(row, len(row)) for row in self.path]

    def reverse(self):
        return [row[::-1] if random.randint(0, 1) == 1 else row for row in self.path]

    # 将部分序列逆序
    def one_exchange(self):
        path = copy.deepcopy(self.path)
        for road in self.path:
            p1 = random.randint(0, len(road) - 1)  # min
            p2 = random.randint(0, len(road) - 1)  # max
            if p1 > p2:
                p1, p2 = p2, p1
            while p1 < p2:
                path[p1], path[p2] = path[p2], path[p1]
                p1 += 1
                p2 -= 1
        return result(self.p, path)

    # 内部交换序列
    def inner_exchange(self):
        path = self.reverse()
        p1 = random.randint(0, len(path) - 1)
        p2 = random.randint(0, len(path) - 1)
        r1 = random.randint(0, len(path[p1]) - 1)
        r2 = random.randint(0, len(path[p2]) - 1)
        if p1 != p2:
            path[p1] = []
            path[p2] = []
            for i in range(r1):
                path[p1].append(self.path[p1][i])
            for i in range(r2):
                path[p2].append(self.path[p2][i])
            for i in range(r1, len(path[p1])):
                path[p2].append(self.path[p1][i])
            for i in range(r2, len(path[p2])):
                path[p1].append(self.path[p2][i])
        return result(self.p, self.adjust(path))

    def outer_exchange(self, res):
        pass

    def adjust(self, path):
        value_new = [5,2,3,6,1]
        # for p in path:
        #     t = 0
        #     for e in p:
        #         t += self.p.demand[e.n1][e.n2]
        #     value_new.append(t)
        # print(value_new)
        # 
        ####################################
        tmp = list(zip(value_new, path))
        path = [x for _, x in sorted(tmp, key=lambda x: x[0])] # key
        ####################################

        wait = []
        for i in range(len(path)):
            path[i].extend(wait)
            value_new[i] += sum(e.demand for e in path[i])
            wait = []
            while value_new[i] > self.p.capacity:
                tmp = random.choice(path[i])
                path[i].remove(tmp)
                wait.append(tmp)
                value_new[i] -= tmp.demand
        if len(wait) != 0:
            value = 0
            e = []
            for r in wait:
                value += r.demand
                if value > self.p.capacity:
                    path.append(e)
                    e = []
                    value = r.demand
                e.append(r)
            if len(e) != 0:
                path.append(e)


class problem():
    def __init__(self, filename):
        self.origin = []
        with open(filename, 'r') as f:
            data = f.readlines()
        self.name = data[0].split(':')[1].strip()
        self.vertices = int(data[1].split(':')[1].strip())
        self.depot = int(data[2].split(':')[1].strip())
        self.required_edges = int(data[3].split(':')[1].strip())
        self.non_required_edges = int(data[4].split(':')[1].strip())
        self.vehicles = int(data[5].split(':')[1].strip())
        self.capacity = int(data[6].split(':')[1].strip())
        self.cost_edges = int(data[7].split(':')[1].strip())
        self.edges = []
        self.target_edges = []
        self.demand = []
        self.solution = []
        global ran_choice_rate
        ran_choice_rate = ran_choice_rate / self.required_edges
        tmp = 0
        self.size = 0
        for i in range(9, len(data)):
            node_data = data[i].split()
            if len(node_data) == 4:
                tmp += 1
                self.size = max(self.size, int(node_data[0]), int(node_data[1]))
                t1 = edge(int(node_data[0]), int(node_data[1]), int(node_data[2]), int(node_data[3]))
                self.edges.append(t1)
                if node_data[3] != 0:
                    self.target_edges.append(t1)
        self.adjacent_matrix = MAX * np.ones((self.size + 1, self.size + 1))
        self.origin_length = np.zeros((self.size + 1, self.size + 1))
        self.demand = np.zeros((self.size + 1, self.size + 1))
        for e in self.edges:
            self.adjacent_matrix[e.n1][e.n2] = e.cost
            self.adjacent_matrix[e.n2][e.n1] = e.cost
            self.origin_length[e.n1][e.n2] = e.cost
            self.origin_length[e.n2][e.n1] = e.cost
            self.demand[e.n1][e.n2] = e.demand
            self.demand[e.n2][e.n1] = e.demand
        self.florid()
        for _ in range(1000):
            self.solution.append(result(self, self.get_root()))
        self.solution.sort(key=lambda e: e.cost)

    def florid(self):
        for i in range(1, self.size + 1):
            self.adjacent_matrix[i][i] = 0
        for k in range(1, self.size + 1):
            for i in range(1, self.size + 1):
                for j in range(1, self.size + 1):
                    if self.adjacent_matrix[i][j] > self.adjacent_matrix[i][k] + self.adjacent_matrix[k][j]:
                        self.adjacent_matrix[i][j] = self.adjacent_matrix[i][k] + self.adjacent_matrix[k][j]

    def get_root(self):
        tmp = self.target_edges.copy()
        now = self.depot
        capacity = self.capacity
        origin = []  # 总路径
        road = []  # 每个车的路径
        while 1:
            p = 0
            if len(tmp) == 0:
                if len(road) != 0:
                    origin.append(road)
                    road = []
                break
            t_edge = None
            range = 0
            for e in tmp:
                r = min(self.adjacent_matrix[e.n1][now], self.adjacent_matrix[e.n2][now])
                if e.demand <= capacity:
                    if t_edge is None:
                        t_edge = e
                        range = r
                    elif p == 0 and (r < range or random.uniform(0, 1) <= ran_choice_rate):
                        t_edge = e
                        p = 1
            if t_edge is None:
                if len(road) != 0:
                    origin.append(road)
                    road = []
                capacity = self.capacity
                now = self.depot
            else:
                road.append(t_edge)
                capacity -= t_edge.demand
                if self.adjacent_matrix[t_edge.n1][now] > self.adjacent_matrix[t_edge.n2][now]:
                    now = t_edge.n1
                else:
                    now = t_edge.n2
                tmp.remove(t_edge)
        return origin

    def get_cost(self, path):
        cost = 0
        tmp = self.depot
        for k in path:
            for p in k:
                if self.adjacent_matrix[tmp][p.n1] > self.adjacent_matrix[tmp][p.n2]:
                    cost += self.adjacent_matrix[tmp][p.n2]
                    tmp = p.n2
                    cost += self.origin_length[tmp][p.n1]
                    tmp = p.n1
                else:
                    cost += self.adjacent_matrix[tmp][p.n1]
                    tmp = p.n1
                    cost += self.origin_length[tmp][p.n2]
                    tmp = p.n2
            cost += self.adjacent_matrix[tmp][self.depot]
            tmp = self.depot
        return cost

    def get_cost1(self, path):
        cost = 0
        tmp = self.depot
        st = ''
        st = st + '0,'
        a = 0
        for k in path:
            if a == 0:
                a = 1
            else:
                st += ',0,'
            for p in k:
                if self.adjacent_matrix[tmp][p.n1] > self.adjacent_matrix[tmp][p.n2]:
                    cost += self.adjacent_matrix[tmp][p.n2]
                    tmp = p.n2
                    st = st + '(' + str(int(tmp)) + ','
                    cost += self.origin_length[tmp][p.n1]
                    tmp = p.n1
                    st = st + str(int(tmp)) + '),'
                else:
                    cost += self.adjacent_matrix[tmp][p.n1]
                    tmp = p.n1
                    st = st + '(' + str(int(tmp)) + ','
                    cost += self.origin_length[tmp][p.n2]
                    tmp = p.n2
                    st = st + str(int(tmp)) + '),'
            st += '0'
            cost += self.adjacent_matrix[tmp][self.depot]
            tmp = self.depot
        return cost, st

    def generation(self, round,reproduce_rate):
        # todo: create detection for break time
        for _ in range(round):
            self.solution = self.genetic(self.solution,reproduce_rate)
        pass

    def genetic(self, answer_list,reproduce_rate):
        new_answer_list = []
        for i in range(100):
            new_answer_list.extend(self.variate(answer_list[i],reproduce_rate))
        return new_answer_list.sort(key=lambda e: e.cost)[:110]

    def variate(self, path,reproduce_rate):
        path_list = [path]
        for _ in range(reproduce_rate):
            path_list.append(path.one_exchange())
            path_list.append(path.inner_exchange())
        return path_list

    def get_answer(self, path):
        cost, str1 = self.get_cost1(path)
        print('s ' + str1)
        print('q ' + str(int(cost)))


def solver(path, termination, seeds):
    p = problem(path)
    p.generation(5,30)
    p.get_answer(p.solution[0].path)
    pass


start = time.time()
# solver = solver(sys.argv[1], int(sys.argv[3]), int(sys.argv[5]))
solver = solver('../datasets/gdb1.dat', 1, 1)


IndexError: Cannot choose from an empty sequence